# Market Clearing Problem - Lecture 1
## Two producers, two consumers and network topology

### Imports

In [1]:
import numpy as np
from pyomo.environ import *

### Defining the world

In [2]:
# Define generators (Max output, offer price, bus location)
generator_defs = ((100,12,1),
                  (80,20,2))

# Define demands (Max consumption, bid price, bus location)
demand_defs = ((100,40,2),
                (50,35,3))

# Define Network
capacity = np.array([[0,100,100],
                     [100,0,100],
                     [100,100,0]])

susceptance = np.array([[0,500,500],
                        [500,0,500],
                        [500,500,0]])

### Setting up optimization problem 

In [3]:
# Create model
model = ConcreteModel()

# Set up generator related objects
model.generators = Set(initialize=range(len(generator_defs))) # Set defining generators
model.PG_max = Param(model.generators,initialize={i:g_[0] for i, g_ in enumerate(generator_defs)}) # Max capacity of generators [MW]
model.C = Param(model.generators,initialize={i:g_[1] for i, g_ in enumerate(generator_defs)}) # Offer price of generators [€/MW]
model.p_G = Var(model.generators,within=NonNegativeReals) # Production of generators [MW]

# Set up demand related objects
model.demands = Set(initialize=range(len(demand_defs))) # Set defining demands
model.L = Param(model.demands,initialize={i:d_[0] for i, d_ in enumerate(demand_defs)}) # Max load of demands [MW]
model.U = Param(model.demands,initialize={i:d_[1] for i, d_ in enumerate(demand_defs)}) # Bid price of demands [€/MW] (U for Utility)
model.p_D = Var(model.demands,within=NonNegativeReals) # Consumption of generators [MW]

# Set up network related objects
model.nodes = Set(initialize=range(len(capacity)))
model.edges = Set(initialize=model.nodes*model.nodes)
model.Fmax = Param(model.nodes,model.nodes,initialize={v_:capacity[v_] for v_ in model.edges})
model.B = Param(model.nodes,model.nodes,initialize={v_:susceptance[v_] for v_ in model.edges})
model.location_generators = Param(model.generators,initialize={i:model.nodes[g_[2]] for i, g_ in enumerate(generator_defs)})
model.location_demands = Param(model.generators,initialize={i:model.nodes[g_[2]] for i, g_ in enumerate(generator_defs)})

# Set up variables for network
model.f = Var(model.nodes,model.nodes) # Power flow from bus to bus [MW]
model.theta = Var(model.nodes) # Voltage angle at bus n [rad.]

# Optimize the social welfare...
def social_welfare(model):
    total_utility = sum(model.U[d_]*model.p_D[d_] for d_ in model.demands)
    total_cost = sum(model.C[g_]*model.p_G[g_] for g_ in model.generators)
    return total_utility-total_cost

model.SW = Objective(rule=social_welfare, sense=maximize)

# ... given that generators has a maximum capacity
def generator_constr_rule(model,i):
    return (0,model.p_G[i],model.PG_max[i])

model.generators_constraint = Constraint(model.generators,rule=generator_constr_rule)

# ... given that consumers has as maximum demand
def demand_constr_rule(model,i):
    return (0,model.p_D[i],model.L[i])

model.demands_constraint = Constraint(model.demands,rule=demand_constr_rule)

# ... given that network topology is respected
def power_flow_constr_rule_1(model,i,j):
    return (-1*model.Fmax[i,j],model.f[i,j],model.Fmax[i,j])

def power_flow_constr_rule_2(model,i,j):
    return model.f[i,j]==model.B[i,j]*(model.theta[i]-model.theta[j])

def power_flow_constr_rule_3(model,i,j):
    return (-1*model.Fmax[i,j],model.B[i,j]*(model.theta[i]-model.theta[j]),model.Fmax[i,j])


model.power_flow_constraint_1 = Constraint(model.nodes,model.nodes,rule=power_flow_constr_rule_1)
model.power_flow_constraint_2 = Constraint(model.nodes,model.nodes,rule=power_flow_constr_rule_2)
model.theta_zero_constraint = Constraint(expr=(model.theta[1],0))


def power_balance_constr_rule(model,i):
    all_gen_at_node = sum([model.p_G[g_] for g_ in model.location_generators if model.location_generators[g_]==i])
    all_dem_at_node = sum([model.p_D[d_] for d_ in model.location_demands if model.location_demands[d_]==i])
    all_phase_diffs_at_node = sum([model.f[v_] for v_ in model.edges if v_[0]==i])
    return -1*all_gen_at_node+all_dem_at_node+all_phase_diffs_at_node==0
    
model.power_balance_at_nodes = Constraint(model.nodes,rule=power_balance_constr_rule)

### Solve

In [4]:
opt = SolverFactory('cbc')
opt.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': -3550.0, 'Upper bound': -3550.0, 'Number of objectives': 1, 'Number of constraints': 40, 'Number of variables': 17, 'Number of nonzeros': 4, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.0, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 1}}, 'Error rc': 0, 'Time': 0.030220746994018555}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Display solution

In [5]:
for g_ in model.generators:
    print('Generator {0} has an offer of {1}€, max capacity of {2}[MW] and produces {3:.2f}[MW]'.format(g_,model.C[g_],model.PG_max[g_],model.p_G[g_].value))
    
print('')

for d_ in model.demands:
    print('Consumer {0} has a bid of {1}€, max consumption of {2}[MW] and loads {3:.2f}[MW]'.format(d_,model.U[d_],model.L[d_],model.p_D[d_].value))

Generator 0 has an offer of 12€, max capacity of 100[MW] and produces 100.00[MW]
Generator 1 has an offer of 20€, max capacity of 80[MW] and produces 50.00[MW]

Consumer 0 has a bid of 40€, max consumption of 100[MW] and loads 100.00[MW]
Consumer 1 has a bid of 35€, max consumption of 50[MW] and loads 50.00[MW]


In [6]:
model.display()

Model unknown

  Variables:
    p_G : Size=2, Index=generators
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 : 100.0 :  None : False : False : NonNegativeReals
          1 :     0 :  50.0 :  None : False : False : NonNegativeReals
    p_D : Size=2, Index=demands
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 : 100.0 :  None : False : False : NonNegativeReals
          1 :     0 :  50.0 :  None : False : False : NonNegativeReals
    f : Size=9, Index=f_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :  None :   0.0 :  None : False : False :  Reals
        (0, 1) :  None :   0.0 :  None : False : False :  Reals
        (0, 2) :  None :   0.0 :  None : False : False :  Reals
        (1, 0) :  None :   0.0 :  None : False : False :  Reals
        (1, 1) :  None :   0.0 :  None : False : False :  Reals
        (1, 2) :  None :   0.0 :  None : False : False :  Reals
        (2, 0) :  Non